### Construct a vertical grid following Stewart et al. (2017)
This script is adapted from the [Stewart et al., 2017](https://github.com/kialstewart/vertical_grid_for_ocean_models/blob/master/build_vertical_grid_kds.py) script to construct a grid used with the MITgcm. The values of `drF` are to be copied to the `input/data` namelist. Additionally, a netcdf file is created for reference.

In [1]:
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

Choose some parameters for the vertical grid

In [2]:
# Maximum depth (meters) of the grid
H = 4000.0

# Maximum grid spacing (meters) in the deep ocean
dzd = 100.0

# Minimum grid spacing (meters) at the surface
min_dz = 1.0

# Sharpness of the hyperbolic tangent (initial value should be adjusted)
depfac = 1.00

# Name of the file to be written that contains the grid
output_name = "vertical_grid.nc"

### Build the grid
First, define the functional form of the vertical grid.

In [3]:
# the function is {tanh(pi*m/H)*dz_max + epsilon}, which is epsilon at the surface and dz_max at H
def f_all(kk, H, depfac, dzd, epsilon):
    return np.tanh(np.pi * ((kk) / (H * depfac))) * (dzd) + epsilon 

This is where the magic happens: an iterative process that takes a step from the current end (deepest point) of the grid along the function to find the next point. After that, we need to relocate the initial grid vertically so that the grid spacing at the surface is min_dz. All this is done in a function so we can interactively change the paraemters later.

In [4]:
# this is a small number needed to begin the iteration
epsilon = 0.001
# Then, make the first two entries of the initial grid; these will be 0 and epsilon for both z and dz.
delta_z = [0, epsilon * 1.0]
prop_z = [0, epsilon * 1.0]
while prop_z[-1] + delta_z[-1] < 1.2 * H:
    aa = np.linspace(1.0, 1.5, 10000)
    bb = np.zeros([len(aa)])
    loopkill = 1.0
    ii = 0
    while loopkill > 0:
        bb[ii] = (f_all(prop_z[-1] + (delta_z[-1] * aa[ii]), H, depfac, dzd, epsilon)) - (delta_z[-1] * aa[ii])
        loopkill = bb[ii]
        ii += 1
    aa_bb = np.polyfit(aa[:ii-1], bb[:ii-1], 1)
    dznew = (delta_z[-1] * (np.abs(aa_bb[1] / aa_bb[0])))
    delta_z = np.append(delta_z, dznew)
    prop_z = np.append(prop_z, (prop_z[-1] + delta_z[-1]))

In [5]:
def con_grid(H, min_dz, depfac, dzd):
    # this is a small number needed to begin the iteration
    epsilon = 0.001
    # Then, make the first two entries of the initial grid; these will be 0 and epsilon for both z and dz.
    delta_z = [0, epsilon * 1.0]
    prop_z = [0, epsilon * 1.0]
    while prop_z[-1] + delta_z[-1] < 1.2 * H:
        aa = np.linspace(1.0, 1.5, 10000)
        bb = np.zeros([len(aa)])
        loopkill = 1.0
        ii = 0
        while loopkill > 0:
            bb[ii] = (f_all(prop_z[-1] + (delta_z[-1] * aa[ii]), H, depfac, dzd, epsilon)) - (delta_z[-1] * aa[ii])
            loopkill = bb[ii]
            ii += 1
        aa_bb = np.polyfit(aa[:ii-1], bb[:ii-1], 1)
        dznew = (delta_z[-1] * (np.abs(aa_bb[1] / aa_bb[0])))
        delta_z = np.append(delta_z, dznew)
        prop_z = np.append(prop_z, (prop_z[-1] + delta_z[-1]))
    # find where the initial grid is min_dz (the surface resolution)
    new_surf = np.max(np.where(delta_z <= min_dz)) 
    # make a new grid that shifts the initial grid vertically
    real_prop_z = prop_z[new_surf:] - prop_z[new_surf] 
    # make a new dz for this new grid
    real_delta_z = delta_z[new_surf + 1:] 
    # cut the new grid off at desired depth, H
    real_prop_z = real_prop_z[np.where(real_prop_z < H)]
    # and the new dz too
    real_delta_z = real_delta_z[np.where(real_prop_z < H)]
    return real_prop_z, real_delta_z

In [6]:
def plot_sub(ax, x, lim, color):
    ax.scatter(x, np.arange(0, -len(x), -1), marker="+", color=color)
    ax.set_ylim(-lim, 0)
    ax.grid()

In [7]:
H_widget = widgets.IntSlider(min=100, max=10000, value=H, step=100)
min_dz_widget = widgets.IntSlider(min=1, max=10, value=min_dz)
dzd_widget = widgets.IntSlider(min=10, max=500, value=dzd, step=10)
depfac_widget = widgets.FloatSlider(min=0.7, max=1.3, value=depfac, step=0.01)
def H_change(v):
    global H
    H = v["new"]
H_widget.observe(H_change, names="value")
def min_dz_change(v):
    global min_dz
    min_dz = v["new"]
min_dz_widget.observe(min_dz_change, names="value")
def dzd_change(v):
    global dzd
    dzd = v["new"]
dzd_widget.observe(dzd_change, names="value")
def depfac_change(v):
    global depfac
    depfac = v["new"]
depfac_widget.observe(depfac_change, names="value")


In [8]:
@interact_manual
def plots(H0=H_widget, min_dz0=min_dz_widget, dzd0=dzd_widget, depfac0=depfac_widget):
    real_prop_z, real_delta_z = con_grid(H0, min_dz0, depfac0, dzd0)
    #
    fig = plt.figure(figsize=(14, 16))
    #
    ax1 = fig.add_subplot(2, 3, 1)
    plot_sub(ax1, real_delta_z, 30, color="tomato")
    ax1.set_title("dz, vertical grid spacing")
    #
    ax2 = fig.add_subplot(2, 3, 2)
    plot_sub(ax2, real_prop_z, 30, color="steelblue")
    ax2.set_title("z, grid levels")
    #
    ax3 = fig.add_subplot(2, 3, 3)
    ax3.scatter(np.zeros(len(real_prop_z)), -real_prop_z, marker="+", color="forestgreen")
    ax3.hlines(-real_prop_z, -1, 1, color="slategrey")
    ax3.set_ylim(-100, 0)
    ax3.set_title("z level positions")
    ax3.set_xticklabels("")
    #
    ax4 = fig.add_subplot(2, 3, 4)
    plot_sub(ax4, real_delta_z, len(real_delta_z), color="tomato")
    ax4.set_title("dz, vertical grid spacing")
    #
    ax5 = fig.add_subplot(2, 3, 5)
    plot_sub(ax5, real_prop_z, len(real_delta_z), color="steelblue")
    ax5.set_title("z, grid levels")
    ax5.set_title("z, grid levels")
    #
    ax6 = fig.add_subplot(2, 3, 6)
    ax6.scatter(np.zeros(len(real_delta_z)), -real_prop_z, marker="+", color="forestgreen")
    ax6.hlines(-real_prop_z, -1, 1, color="slategrey")
    ax6.set_title("z level positions")
    ax6.set_xticklabels("")
    #
    fig.suptitle("Adjusted Grid", fontsize=20);
    fig.subplots_adjust(top=0.93)
    print("The number of levels is ", str(len(real_delta_z)))
    print("The surface level is ", str(real_delta_z[0]), "m thick")
    print("The lowest level is ", str(real_delta_z[-1]), "m thick")
    print("The maximum depth is ", str(real_prop_z[-1]), "m")

interactive(children=(IntSlider(value=4000, description='H0', max=10000, min=100, step=100), IntSlider(value=1…

Based on which parameters we have chosen in the interactive panel, we now calculate the vertical axis and all the different properties as they would appear in the MITgcm grid output.

In [9]:
final_prop_z, final_delta_z = con_grid(H, min_dz, depfac, dzd)

In [10]:
Z = -(final_prop_z + final_delta_z/2)
Zp1 = -np.append(final_prop_z[:], final_prop_z[-1] + final_delta_z[-1])
Zu = -np.append(final_prop_z[1::], final_prop_z[-1] + final_delta_z[-1])
Zl = -final_prop_z
drC = np.append(np.append(0 - Z[0], Z[0:-1] - Z[1::]), -Zu[-1] + Z[-1])
drF = final_delta_z

In [11]:
ds_out = xr.Dataset(coords={"Z": (["Z"], Z),
                            "Zp1": (["Zp1"], Zp1),
                            "Zu": (["Zu"], Zu),
                            "Zl": (["Zl"], Zl),
                            "drC": (["Zp1"], drC),
                            "drF": (["Z"], drF),
                            })
ds_out["Z"].attrs = {"standard_name": "depth", 
                     "long_name": "vertical coordinate of cell center",
                     "units": "m",
                     "positive": "up",
                     "axis": "Z"}
ds_out["Zp1"].attrs = {"standard_name": "depth_at_w_location", 
                       "long_name": "vertical coordinate of cell interface",
                       "units": "m",
                       "positive": "up",
                       "axis": "Z",
                       "c_grid_axis_shift": (-0.5, 0.5)}
ds_out["Zu"].attrs = {"standard_name": "depth_at_lower_w_location", 
                      "long_name": "vertical coordinate of lower cell interface",
                      "units": "m",
                      "positive": "up",
                      "axis": "Z",
                      "c_grid_axis_shift": 0.5}
ds_out["Zl"].attrs = {"standard_name": "depth_at_upper_w_location", 
                      "long_name": "vertical coordinate of upper cell interface",
                      "units": "m",
                      "positive": "up",
                      "axis": "Z",
                      "c_grid_axis_shift": -0.5}
ds_out["drC"].attrs = {"standard_name": "cell_z_size_at_w_location", 
                       "long_name": "cell z size",
                       "units": "m"}
ds_out["drF"].attrs = {"standard_name": "cell_z_size", 
                       "long_name": "cell z size",
                       "units": "m"}
ds_out.attrs = {"input maximum depth H": str(H),
                "actual maximum depth": str(-np.min(Zp1)),
                "input minimum grid spacing min_dz": str(min_dz),
                "actual minimum grid spacing": str(np.min(drF)),
                "input maximum grid spazing dzd": str(dzd),
                "actual maximum grid spacing": str(np.max(drF)),
                "input factor depfac": str(depfac)}


In [12]:
ds_out.to_netcdf(output_name)